# Modelado

In [1]:
# import libraries
import pandas as pd
from optimus import Optimus
op = Optimus()

In [33]:
# Read the data
# Data from http://rpubs.com/rhuebner/HRCodebook-13
df = op.read.csv("data/hr-data.csv", header=True)

In [34]:
# See the data
df.table()

Employee_Name 1 (string) nullable,EmpID 2 (string) nullable,MarriedID 3 (string) nullable,MaritalStatusID 4 (string) nullable,GenderID 5 (string) nullable,EmpStatusID 6 (string) nullable,DeptID 7 (string) nullable,PerfScoreID 8 (string) nullable,FromDiversityJobFairID 9 (string) nullable,PayRate 10 (string) nullable,Termd 11 (string) nullable,PositionID 12 (string) nullable,Position 13 (string) nullable,State 14 (string) nullable,Zip 15 (string) nullable,DOB 16 (string) nullable,Sex 17 (string) nullable,MaritalDesc 18 (string) nullable,CitizenDesc 19 (string) nullable,HispanicLatino 20 (string) nullable,RaceDesc 21 (string) nullable,DateofHire 22 (string) nullable,DateofTermination 23 (string) nullable,TermReason 24 (string) nullable,EmploymentStatus 25 (string) nullable,Department 26 (string) nullable,ManagerName 27 (string) nullable,ManagerID 28 (string) nullable,RecruitmentSource 29 (string) nullable,PerformanceScore 30 (string) nullable,EngagementSurvey 31 (string) nullable,EmpSatisfaction 32 (string) nullable,SpecialProjectsCount 33 (string) nullable,LastPerformanceReview_Date 34 (string) nullable,DaysLateLast30 35 (string) nullable
"Brown,⋅Mia",1103024456,1,1,0,1,1,3,1,28.50,0,1,Accountant⋅I,MA,01450,11/24/87,F,Married,US⋅Citizen,No,Black⋅or⋅African⋅American,10/27/2008,None,N/A⋅-⋅still⋅employed,Active,Admin⋅Offices,Brandon⋅R.⋅LeBlanc,1,Diversity⋅Job⋅Fair,Fully⋅Meets,2.04,2,6,1/15/2019,0
"LaRotonda,⋅William⋅⋅",1106026572,0,2,1,1,1,3,0,23.00,0,1,Accountant⋅I,MA,01460,04/26/84,M⋅,Divorced,US⋅Citizen,No,Black⋅or⋅African⋅American,1/6/2014,None,N/A⋅-⋅still⋅employed,Active,Admin⋅Offices,Brandon⋅R.⋅LeBlanc,1,Website⋅Banner⋅Ads,Fully⋅Meets,5.00,4,4,1/17/2019,0
"Steans,⋅Tyrone⋅⋅",1302053333,0,0,1,1,1,3,0,29.00,0,1,Accountant⋅I,MA,02703,09/01/86,M⋅,Single,US⋅Citizen,No,White,9/29/2014,None,N/A⋅-⋅still⋅employed,Active,Admin⋅Offices,Brandon⋅R.⋅LeBlanc,1,Internet⋅Search,Fully⋅Meets,3.90,5,5,1/18/2019,0
"Howard,⋅Estelle",1211050782,1,1,0,1,1,3,0,21.50,1,2,Administrative⋅Assistant,MA,02170,09/16/85,F,Married,US⋅Citizen,No,White,2/16/2015,04/15/15,None,Terminated⋅for⋅Cause,Admin⋅Offices,Brandon⋅R.⋅LeBlanc,1,Pay⋅Per⋅Click⋅-⋅Google,Fully⋅Meets,3.24,3,4,None,None
"Singh,⋅Nan⋅",1307059817,0,0,0,1,1,3,0,16.56,0,2,Administrative⋅Assistant,MA,02330,05/19/88,F,Single,US⋅Citizen,No,White,5/1/2015,None,N/A⋅-⋅still⋅employed,Active,Admin⋅Offices,Brandon⋅R.⋅LeBlanc,1,Website⋅Banner⋅Ads,Fully⋅Meets,5.00,3,5,1/15/2019,0
"Smith,⋅Leigh⋅Ann",711007713,1,1,0,5,1,3,1,20.50,1,2,Administrative⋅Assistant,MA,01844,06/14/87,F,Married,US⋅Citizen,No,Asian,9/26/2011,09/25/13,career⋅change,Voluntarily⋅Terminated,Admin⋅Offices,Brandon⋅R.⋅LeBlanc,1,Diversity⋅Job⋅Fair,Fully⋅Meets,3.80,4,4,None,None
"Bunbury,⋅Jessica",1504073368,1,1,0,5,6,3,0,55.00,1,3,Area⋅Sales⋅Manager,VA,21851,06/01/64,F,Married,Eligible⋅NonCitizen,No,Black⋅or⋅African⋅American,8/15/2011,08/02/14,Another⋅position,Voluntarily⋅Terminated,Sales,John⋅Smith,17,Social⋅Networks⋅-⋅Facebook⋅Twitter⋅etc,Fully⋅Meets,3.14,5,0,None,None
"Carter,⋅Michelle⋅",1403065721,0,0,0,1,6,3,0,55.00,0,3,Area⋅Sales⋅Manager,VT,05664,05/15/63,F,Single,US⋅Citizen,No,White,8/18/2014,None,N/A⋅-⋅still⋅employed,Active,Sales,John⋅Smith,17,Billboard,Fully⋅Meets,5.00,5,0,1/21/2019,0
"Dietrich,⋅Jenna⋅⋅",1408069481,0,0,0,1,6,1,0,55.00,0,3,Area⋅Sales⋅Manager,WA,98052,05/14/87,F,Single,US⋅Citizen,Yes,White,2/20/2012,None,N/A⋅-⋅still⋅employed,Active,Sales,John⋅Smith,17,Website⋅Banner⋅Ads,PIP,2.30,1,0,1/29/2019,0
"Digitale,⋅Alfred",1306059197,1,1,1,1,6,3,0,56.00,0,3,Area⋅Sales⋅Manager,NH,03062,09/14/88,M⋅,Married,US⋅Citizen,No,American⋅Indian⋅or⋅Alaska⋅Native,8/18/2014,None,N/A⋅-⋅still⋅employed,Active,Sales,John⋅Smith,17,Pay⋅Per⋅Click⋅-⋅Google,Fully⋅Meets,3.60,5,0,1/30/2019,0


In [35]:
# Basic data cleaning
from pyspark.sql.functions import when, count, col, isnull


integer_cols = ["MarriedID","MaritalStatusID", "EmpStatusID", "DeptID", "PerfScoreID", "PositionID", "Termd", "ManagerID", 
                "EmpSatisfaction", "SpecialProjectsCount", "DaysLateLast30", "GenderID"]
float_cols = ["PayRate", "EngagementSurvey"]

for col_name in integer_cols:
    df = df.withColumn(col_name, col(col_name).cast('int'))
    
for col_name in float_cols:
    df = df.withColumn(col_name, col(col_name).cast('float'))
    
df = df.dropna(how="all")
df = df.cols.years_between("DOB", date_format="mm/dd/yy",output_cols="Age")

from pyspark.ml.feature import Bucketizer
splits = [10, 20, 30, 40, 50, 60, float("inf")]

bucketizer = Bucketizer(splits=splits, inputCol="Age", outputCol="Age_bucket")
df = bucketizer.transform(df)

In [36]:
# Número de datos faltantes por columna
from pyspark.sql.functions import when, count, col, isnull

df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).table()

Employee_Name 1 (bigint) not nullable,EmpID 2 (bigint) not nullable,MarriedID 3 (bigint) not nullable,MaritalStatusID 4 (bigint) not nullable,GenderID 5 (bigint) not nullable,EmpStatusID 6 (bigint) not nullable,DeptID 7 (bigint) not nullable,PerfScoreID 8 (bigint) not nullable,FromDiversityJobFairID 9 (bigint) not nullable,PayRate 10 (bigint) not nullable,Termd 11 (bigint) not nullable,PositionID 12 (bigint) not nullable,Position 13 (bigint) not nullable,State 14 (bigint) not nullable,Zip 15 (bigint) not nullable,DOB 16 (bigint) not nullable,Sex 17 (bigint) not nullable,MaritalDesc 18 (bigint) not nullable,CitizenDesc 19 (bigint) not nullable,HispanicLatino 20 (bigint) not nullable,RaceDesc 21 (bigint) not nullable,DateofHire 22 (bigint) not nullable,DateofTermination 23 (bigint) not nullable,TermReason 24 (bigint) not nullable,EmploymentStatus 25 (bigint) not nullable,Department 26 (bigint) not nullable,ManagerName 27 (bigint) not nullable,ManagerID 28 (bigint) not nullable,RecruitmentSource 29 (bigint) not nullable,PerformanceScore 30 (bigint) not nullable,EngagementSurvey 31 (bigint) not nullable,EmpSatisfaction 32 (bigint) not nullable,SpecialProjectsCount 33 (bigint) not nullable,LastPerformanceReview_Date 34 (bigint) not nullable,DaysLateLast30 35 (bigint) not nullable,Age 36 (bigint) not nullable,Age_bucket 37 (bigint) not nullable
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,207,1,0,0,0,8,0,0,0,0,0,103,103,0,0


In [37]:
df = df.dropna(how="any", subset=["ManagerID"])

In [39]:
# Número de datos faltantes por columna
from pyspark.sql.functions import when, count, col, isnull

df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).table()

Employee_Name 1 (bigint) not nullable,EmpID 2 (bigint) not nullable,MarriedID 3 (bigint) not nullable,MaritalStatusID 4 (bigint) not nullable,GenderID 5 (bigint) not nullable,EmpStatusID 6 (bigint) not nullable,DeptID 7 (bigint) not nullable,PerfScoreID 8 (bigint) not nullable,FromDiversityJobFairID 9 (bigint) not nullable,PayRate 10 (bigint) not nullable,Termd 11 (bigint) not nullable,PositionID 12 (bigint) not nullable,Position 13 (bigint) not nullable,State 14 (bigint) not nullable,Zip 15 (bigint) not nullable,DOB 16 (bigint) not nullable,Sex 17 (bigint) not nullable,MaritalDesc 18 (bigint) not nullable,CitizenDesc 19 (bigint) not nullable,HispanicLatino 20 (bigint) not nullable,RaceDesc 21 (bigint) not nullable,DateofHire 22 (bigint) not nullable,DateofTermination 23 (bigint) not nullable,TermReason 24 (bigint) not nullable,EmploymentStatus 25 (bigint) not nullable,Department 26 (bigint) not nullable,ManagerName 27 (bigint) not nullable,ManagerID 28 (bigint) not nullable,RecruitmentSource 29 (bigint) not nullable,PerformanceScore 30 (bigint) not nullable,EngagementSurvey 31 (bigint) not nullable,EmpSatisfaction 32 (bigint) not nullable,SpecialProjectsCount 33 (bigint) not nullable,LastPerformanceReview_Date 34 (bigint) not nullable,DaysLateLast30 35 (bigint) not nullable,Age 36 (bigint) not nullable,Age_bucket 37 (bigint) not nullable
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,199,1,0,0,0,0,0,0,0,0,0,103,103,0,0


In [40]:
from optimus.ml import feature as fe
df = fe.vector_assembler(df,input_cols=['MaritalStatusID', 'GenderID', 'DeptID', 'PayRate', "ManagerID", 
                                   'EmpSatisfaction', 'Age_bucket'], output_col="features")

In [41]:
from pyspark.ml.feature import ChiSqSelector

selector = ChiSqSelector(numTopFeatures=5, featuresCol="features",
                         outputCol="selected_features", labelCol="Termd")

result = selector.fit(df).transform(df)

In [45]:
result.table()

Employee_Name 1 (string) nullable,EmpID 2 (string) nullable,MarriedID 3 (int) nullable,MaritalStatusID 4 (int) nullable,GenderID 5 (int) nullable,EmpStatusID 6 (int) nullable,DeptID 7 (int) nullable,PerfScoreID 8 (int) nullable,FromDiversityJobFairID 9 (string) nullable,PayRate 10 (float) nullable,Termd 11 (int) nullable,PositionID 12 (int) nullable,Position 13 (string) nullable,State 14 (string) nullable,Zip 15 (string) nullable,DOB 16 (string) nullable,Sex 17 (string) nullable,MaritalDesc 18 (string) nullable,CitizenDesc 19 (string) nullable,HispanicLatino 20 (string) nullable,RaceDesc 21 (string) nullable,DateofHire 22 (string) nullable,DateofTermination 23 (string) nullable,TermReason 24 (string) nullable,EmploymentStatus 25 (string) nullable,Department 26 (string) nullable,ManagerName 27 (string) nullable,ManagerID 28 (int) nullable,RecruitmentSource 29 (string) nullable,PerformanceScore 30 (string) nullable,EngagementSurvey 31 (float) nullable,EmpSatisfaction 32 (int) nullable,SpecialProjectsCount 33 (int) nullable,LastPerformanceReview_Date 34 (string) nullable,DaysLateLast30 35 (int) nullable,Age 36 (float) nullable,Age_bucket 37 (double) nullable,features 38 (vector) nullable,selected_features 39 (vector) nullable
"Brown,⋅Mia",1103024456,1,1,0,1,1,3,1,28.5,0,1,Accountant⋅I,MA,01450,11/24/87,F,Married,US⋅Citizen,No,Black⋅or⋅African⋅American,10/27/2008,None,N/A⋅-⋅still⋅employed,Active,Admin⋅Offices,Brandon⋅R.⋅LeBlanc,1,Diversity⋅Job⋅Fair,Fully⋅Meets,2.0399999618530273,2,6,1/15/2019,0,32.779598236083984,2.0,"[1.0,0.0,1.0,28.5,1.0,2.0,2.0]","[1.0,1.0,28.5,1.0,2.0]"
"LaRotonda,⋅William⋅⋅",1106026572,0,2,1,1,1,3,0,23.0,0,1,Accountant⋅I,MA,01460,04/26/84,M⋅,Divorced,US⋅Citizen,No,Black⋅or⋅African⋅American,1/6/2014,None,N/A⋅-⋅still⋅employed,Active,Admin⋅Offices,Brandon⋅R.⋅LeBlanc,1,Website⋅Banner⋅Ads,Fully⋅Meets,5.0,4,4,1/17/2019,0,35.774200439453125,2.0,"[2.0,1.0,1.0,23.0,1.0,4.0,2.0]","[2.0,1.0,23.0,1.0,2.0]"
"Steans,⋅Tyrone⋅⋅",1302053333,0,0,1,1,1,3,0,29.0,0,1,Accountant⋅I,MA,02703,09/01/86,M⋅,Single,US⋅Citizen,No,White,9/29/2014,None,N/A⋅-⋅still⋅employed,Active,Admin⋅Offices,Brandon⋅R.⋅LeBlanc,1,Internet⋅Search,Fully⋅Meets,3.9000000953674316,5,5,1/18/2019,0,33.841400146484375,2.0,"[0.0,1.0,1.0,29.0,1.0,5.0,2.0]","[0.0,1.0,29.0,1.0,2.0]"
"Howard,⋅Estelle",1211050782,1,1,0,1,1,3,0,21.5,1,2,Administrative⋅Assistant,MA,02170,09/16/85,F,Married,US⋅Citizen,No,White,2/16/2015,04/15/15,None,Terminated⋅for⋅Cause,Admin⋅Offices,Brandon⋅R.⋅LeBlanc,1,Pay⋅Per⋅Click⋅-⋅Google,Fully⋅Meets,3.240000009536743,3,4,None,None,34.80110168457031,2.0,"[1.0,0.0,1.0,21.5,1.0,3.0,2.0]","[1.0,1.0,21.5,1.0,2.0]"
"Singh,⋅Nan⋅",1307059817,0,0,0,1,1,3,0,16.559999465942383,0,2,Administrative⋅Assistant,MA,02330,05/19/88,F,Single,US⋅Citizen,No,White,5/1/2015,None,N/A⋅-⋅still⋅employed,Active,Admin⋅Offices,Brandon⋅R.⋅LeBlanc,1,Website⋅Banner⋅Ads,Fully⋅Meets,5.0,3,5,1/15/2019,0,31.792999267578125,2.0,"[0.0,0.0,1.0,16.559999465942383,1.0,3.0,2.0]","[0.0,1.0,16.559999465942383,1.0,2.0]"
"Smith,⋅Leigh⋅Ann",711007713,1,1,0,5,1,3,1,20.5,1,2,Administrative⋅Assistant,MA,01844,06/14/87,F,Married,US⋅Citizen,No,Asian,9/26/2011,09/25/13,career⋅change,Voluntarily⋅Terminated,Admin⋅Offices,Brandon⋅R.⋅LeBlanc,1,Diversity⋅Job⋅Fair,Fully⋅Meets,3.799999952316284,4,4,None,None,32.80649948120117,2.0,"[1.0,0.0,1.0,20.5,1.0,4.0,2.0]","[1.0,1.0,20.5,1.0,2.0]"
"Bunbury,⋅Jessica",1504073368,1,1,0,5,6,3,0,55.0,1,3,Area⋅Sales⋅Manager,VA,21851,06/01/64,F,Married,Eligible⋅NonCitizen,No,Black⋅or⋅African⋅American,8/15/2011,08/02/14,Another⋅position,Voluntarily⋅Terminated,Sales,John⋅Smith,17,Social⋅Networks⋅-⋅Facebook⋅Twitter⋅etc,Fully⋅Meets,3.140000104904175,5,0,None,None,55.841400146484375,4.0,"[1.0,0.0,6.0,55.0,17.0,5.0,4.0]","[1.0,6.0,55.0,17.0,4.0]"
"Carter,⋅Michelle⋅",1403065721,0,0,0,1,6,3,0,55.0,0,3,Area⋅Sales⋅Manager,VT,05664,05/15/63,F,Single,US⋅Citizen,No,White,8/18/2014,None,N/A⋅-⋅still⋅employed,Active,Sales,John⋅Smith,17,Billboard,Fully⋅Meets,5.0,5,0,1/21/2019,0,56.80379867553711

In [48]:
# Train model.  This also runs the indexers.
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol="Termd", featuresCol="selected_features", maxIter=10)
model = gbt.fit(result)

In [49]:
# Make predictions.
predictions = model.transform(result)

In [51]:
# Select example rows to display.
predictions.select("prediction", "Termd", "selected_features").table()

prediction 1 (double) not nullable,Termd 2 (int) nullable,selected_features 3 (vector) nullable
0.0,0,"[1.0,1.0,28.5,1.0,2.0]"
0.0,0,"[2.0,1.0,23.0,1.0,2.0]"
0.0,0,"[0.0,1.0,29.0,1.0,2.0]"
1.0,1,"[1.0,1.0,21.5,1.0,2.0]"
0.0,0,"[0.0,1.0,16.559999465942383,1.0,2.0]"
1.0,1,"[1.0,1.0,20.5,1.0,2.0]"
1.0,1,"[1.0,6.0,55.0,17.0,4.0]"
0.0,0,"[0.0,6.0,55.0,17.0,4.0]"
0.0,0,"[0.0,6.0,55.0,17.0,2.0]"
0.0,0,"[1.0,6.0,56.0,17.0,2.0]"


In [53]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="Termd", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.109272


In [54]:
accuracy

0.890728476821192

## Mejorando el modelo

In [55]:
# División de datos
(training_data, test_data) = result.randomSplit([0.7, 0.3])

In [56]:
# Train model.  This also runs the indexers.
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol="Termd", featuresCol="selected_features", maxIter=10)
model = gbt.fit(training_data)

# Make predictions.
predictions = model.transform(test_data)

In [57]:
# Select example rows to display.
predictions.select("prediction", "Termd", "selected_features").table()

prediction 1 (double) not nullable,Termd 2 (int) nullable,selected_features 3 (vector) nullable
0.0,0,"[2.0,5.0,54.5,2.0,4.0]"
1.0,1,"[2.0,5.0,16.0,39.0,2.0]"
1.0,0,"[0.0,4.0,47.599998474121094,10.0,3.0]"
0.0,1,"[1.0,5.0,22.0,20.0,3.0]"
0.0,0,"[0.0,5.0,27.0,22.0,1.0]"
0.0,1,"[0.0,5.0,26.0,16.0,4.0]"
0.0,1,"[0.0,5.0,50.5,2.0,3.0]"
0.0,1,"[1.0,5.0,20.0,14.0,1.0]"
1.0,0,"[0.0,5.0,25.0,11.0,2.0]"
0.0,0,"[0.0,4.0,56.0,10.0,2.0]"


In [58]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="Termd", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.271739


In [60]:
model.featureImportances

SparseVector(5, {0: 0.158, 1: 0.0889, 2: 0.3095, 3: 0.2715, 4: 0.1722})